In [17]:
import os
import pydot

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import Simulator, StartMeshcat, DiagramBuilder, MeshcatVisualizer

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [18]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7005


In [19]:
def make_stack():
    output = ''
    num_layers = 18 # actual number is 18, but use a smaller number to speed up sim setup
    num_blocks_per_layer = 3
    block_width = 0.025
    block_height = 0.015
    for i in range(num_layers):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(num_blocks_per_layer):
            x = (block_width * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (block_width * (j - 1))
            z = block_height * (i + 0.5)
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output

In [20]:
def create_scene():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_link
"""

    robot_data = """
directives:
- add_model:
    name: iiwa_left
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa_left::iiwa_link_0
- add_model:
    name: wsg_left
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa_left::iiwa_link_7
    child: wsg_left::body
    X_PC:
        translation: [-0.6, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: iiwa_right
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa_right::iiwa_link_0
    X_PC:
        translation: [.6, 0, 0]
        rotation: !Rpy { deg: [0, 0, 0]}
- add_model:
    name: wsg_right
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa_right::iiwa_link_7
    child: wsg_right::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
model_drivers:
    iiwa_left: !IiwaDriver
      hand_model_name: wsg_left
    wsg_left: !SchunkWsgDriver {}
    iiwa_right: !IiwaDriver
      hand_model_name: wsg_right
    wsg_right: !SchunkWsgDriver {}
"""

    camera_data = """
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy { deg: [0, 0, 0]}
            translation: [0, 1, 1]
    cameras:
        camera0:
        name: camera0
        depth: True
        X_PB:
            base_frame: camera0_origin
    """

    scenario_data += make_stack()
    scenario_data += robot_data
    # scenario_data += camera_data
    print(scenario_data)
    
    scenario = load_scenario(data=scenario_data)
    builder = DiagramBuilder()
    
    #station = MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml'])
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml']))
    #context = station.CreateDefaultContext()
    
    
    plant = station.GetSubsystemByName("plant")
    controller_plant = station.GetSubsystemByName(
        "iiwa_left.controller"
    ).get_multibody_plant_for_control()
    
    
    #station.GetInputPort("wsg.position").FixValue(context, [0.1])
    #station.ForcedPublish(context)  
    
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat
    )

    diagram= builder.Build()
    
    display(
    SVG(
        pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=1))[
            0
        ].create_svg()
    )
)

    simulator = Simulator(station)
    
    context = simulator.get_mutable_context()
    station_context = station.GetMyMutableContextFromRoot(context)
    plant_context = plant.GetMyMutableContextFromRoot(context)

    q0 = plant.GetPositions(
        plant_context, plant.GetModelInstanceByName("iiwa_left")
    )
    station.GetInputPort("iiwa_left.position").FixValue(station_context, q0)
    station.GetInputPort("iiwa_right.position").FixValue(station_context, q0)

    station.GetInputPort("wsg_left.position").FixValue(station_context, [0])
    station.GetInputPort("wsg_right.position").FixValue(station_context, [0])
    
    
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


create_scene()


directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_link

- add_model:
    name: block_0
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [-0.025, 0, 0.0075]
            rotation: !Rpy { deg: [0, 0, 0] }
- add_model:
    name: block_1
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [0.0, 0, 0.0075]
            rotation: !Rpy { deg: [0, 0, 0] }
- add_model:
    name: block_2
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [0.025, 0, 0.0075]
            rotation: !Rpy { deg: [0, 0, 0] }
- add_model:
    name: block_3
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            transla